In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import skimage
import os
from sklearn.metrics import mean_squared_error
from skimage import data, img_as_float
from skimage.restoration import denoise_nl_means
from skimage.measure import compare_ssim as ssim
from random import shuffle
import math
from keras.models import model_from_json

from utilities import *
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.layers import *
from keras.models import Model
from keras import regularizers
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.layers.core import Lambda

In [3]:
def read_all_datasets():
    rawimages = read_dataset('data/all-mias/')
    rawimages.extend(read_dataset('data/dental'))
    rawimages.extend(read_dataset('data/RawImage/TrainingData'))
    return np.array(rawimages)

In [4]:
rawimages = read_all_datasets()
rawimages.shape

(722, 64, 64)

In [5]:
#preprocessing
rawimages = rawimages.astype('float32')/255.
images = np.reshape(rawimages, (rawimages.shape[0],64,64,1))

In [6]:
#Adding noise to images
def add_noise(images):
    #x_train = np.reshape(x_train, (len(x_train), 64*64))  # adapt this if using `channels_first` image data format
    #x_test = np.reshape(x_test, (len(x_test), 64*64))  # adapt this if using `channels_first` image data format
    batch = images.shape[0]/4;
    noise1 = gaussian_noise(images[0:batch],0,1,0.1)
    noise2 = gaussian_noise(images[batch:2*batch],0,1,0.5)
    noise3 = gaussian_noise(images[2*batch:3*batch],0,2,0.2)
    noise4 = gaussian_noise(images[3*batch:],0,3,0.1)
    
    noisy_set = []
    for data in [noise1,noise2,noise3,noise4]:
        for image in data:
            noisy_set.append(image)
    
    return np.array(noisy_set)
   
#Shuffle the noisy image ground truth pair to randomize the noise distribution in the dataset
def pair_shuffle(images,noisy_set):
    image_pair = []
    for i in range(images.shape[0]):
        image_pair.append((images[i],noisy_set[i]))
    shuffle(image_pair)
    
    ground_truth=[]
    noisy_images = []
    for i in range(images.shape[0]):
        ground_truth.append(image_pair[i][0])
        noisy_images.append(image_pair[i][1])
    return np.array(ground_truth), np.array(noisy_images)

In [7]:
#Shuffling and adding noise to the dataset
shuffle(images)
#Getting the noisy image set
noisy_set = add_noise(images)
ground_truth,noisy_images = pair_shuffle(images,noisy_set)

In [8]:
#Split into training and cross validation
train_size = int(ground_truth.shape[0]*0.8)
x_train = ground_truth[0:train_size]
x_train_noisy = noisy_images[0:train_size]
x_test = ground_truth[train_size:]
x_test_noisy = noisy_images[train_size:]
print x_train_noisy.shape
print x_test_noisy.shape

(577, 64, 64, 1)
(145, 64, 64, 1)


In [9]:
#Ground truth
display_images(ground_truth,10)

#Noisy images
display_images(noisy_images,10)

RuntimeError: Could not create write struct

RuntimeError: Could not create write struct

In [10]:
#Defining the model

def get_simple_autoencoder_model(model_path=None):
    
    if(model_path is None):
        autoencoder = None
    else:
        autoencoder = read_model_json(model_path) 
    
    if(autoencoder is None):
        input_img = Input(shape=x_train_noisy[0].shape)  # adapt this if using `channels_first` image data format

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(input_img)
        x = MaxPooling2D((2, 2), padding='same')(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        encoded = MaxPooling2D((2, 2), padding='same')(x)

        x = Conv2D(64, (3, 3), activation='relu', padding='same')(encoded)
        x = UpSampling2D((2, 2))(x)
        x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
        x = UpSampling2D((2, 2))(x)
        decoded = Conv2D(1, (3, 3), activation='sigmoid', padding='same')(x)

        autoencoder = Model(input_img, decoded)
        autoencoder.compile(optimizer='adadelta', loss='binary_crossentropy',metrics = ['accuracy'])

    print (autoencoder.summary())
    return autoencoder

In [11]:
#Training the model
autoencoder = get_simple_autoencoder_model()
autoencoder.fit(x_train_noisy, x_train,
                epochs=1,
                batch_size=10,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', histogram_freq=0, write_graph=True)])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 64, 64, 1)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 64)        640       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 32, 32, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 16, 16, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 64)        36928     
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 32, 32, 64)        0         
__________

In [12]:
#Function to get saved keras model
def read_model_json(jsonfilePath,h5filePath):
    try:
        json_file = open(jsonfilePath, 'r')
        print json_file
        loaded_model_json = json_file.read()
        json_file.close()
        print "hello"
        loaded_model = model_from_json(loaded_model_json)
         
        # load weights into new model
        loaded_model.load_weights(h5filePath)

        return loaded_model
    except:
        return None

In [13]:
def get_gated_connections(gatePercentageFactor,inputLayer):
    gateFactor = Input(tensor = K.variable([gatePercentageFactor]))
    fractionG = Lambda(lambda x: x[0]*x[1])([inputLayer,gateFactor])
    complement = Lambda(lambda x: x[0] - x[1])([inputLayer,fractionG])
    
    return gateFactor,fractionG,complement

#x is conv layer
#y is de-conv layer
#gf is gating factor
#fg is fractional input from gate
#c is complement ie remaining fraction from the gate
#jt joining tensor of convolution layer and previous de-conv layer 

def get_cnn_dsc_architecture(model_path=None):
    
    if(model_path is None):
        sym_autoencoder = None
    else:
        sym_autoencoder = read_model_json(model_path[0],model_path[1])
        print model_path[0],model_path[1]
    if(sym_autoencoder is None):
        input_img = Input(shape=(64,64,1))  # adapt this if using `channels_first` image data format
        x1 = Conv2D(64, (4, 4), activation='relu', padding='same')(input_img)
        gf1,fg1,c1 = get_gated_connections(0.1,x1)

        x = MaxPooling2D((2, 2), padding='same')(fg1)
        x2 = Conv2D(64, (4, 4), activation='relu', padding='same')(x) 
        gf2,fg2,c2 = get_gated_connections(0.2,x2)

        x = MaxPooling2D((2, 2), padding='same')(fg2)
        x3 = Conv2D(128, (4, 4), activation='relu', padding='same')(x) 
        gf3,fg3,c3 = get_gated_connections(0.3,x3)

        x = MaxPooling2D((2, 2), padding='same')(x3)
        x4 = Conv2D(256, (4, 4), activation='relu', padding='same')(x) 
        gf4,fg4,c4 = get_gated_connections(0.4,x4)

        x = MaxPooling2D((2, 2), padding='same')(x4)
        x5 = Conv2D(512, (4, 4), activation='relu', padding='same')(x) 

        x = UpSampling2D((2, 2))(x5)
        y1 = Conv2DTranspose(256, (4, 4), activation='relu', padding='same')(x) 
        jt4 = Add()([y1,c4])
        x = UpSampling2D((2, 2))(jt4)

        y2 = Conv2DTranspose(128, (4, 4), activation='relu', padding='same')(x) 
        jt3 = Add()([y2,c3])
        x = UpSampling2D((2, 2))(jt3)

        y3 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(x) 
        jt2 = Add()([y3,c2])
        x = UpSampling2D((2, 2))(jt2)

        jt1 = Add()([x,c1])
        y4 = Conv2DTranspose(64, (4, 4), activation='relu', padding='same')(jt1)
        y5 = Conv2DTranspose(1, (4, 4), activation='relu', padding='same')(y4) 

        layers = y5

        sym_autoencoder = Model([input_img,gf1,gf2,gf3,gf4],layers)
        sym_autoencoder.compile(optimizer='sgd', loss = 'mean_squared_error', metrics = ['accuracy'])
        print "Model created"
    else:
        print "Saved model loaded"
    print sym_autoencoder.summary()
    return sym_autoencoder

In [14]:
sym_autoencoder = get_cnn_dsc_architecture()

Model created
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, 64, 64, 1)     0                                            
____________________________________________________________________________________________________
conv2d_6 (Conv2D)                (None, 64, 64, 64)    1088        input_2[0][0]                    
____________________________________________________________________________________________________
input_3 (InputLayer)             (1,)                  0                                            
____________________________________________________________________________________________________
lambda_1 (Lambda)                (None, 64, 64, 64)    0           conv2d_6[0][0]                   
                                                                   input_3[0]

In [15]:
sym_autoencoder.fit(x_train_noisy, x_train,
                epochs=1,
                batch_size=10,
                shuffle=True,
                validation_data=(x_test_noisy, x_test),
                callbacks=[TensorBoard(log_dir='/tmp/autoencoder', 
                                       histogram_freq=0,
                                       write_graph=True)])

Train on 577 samples, validate on 145 samples
Epoch 1/1
577/577 [==============================] - 135s - loss: 0.0365 - acc: 0.3818 - val_loss: 0.0196 - val_acc: 0.3772


In [16]:
x_train_noisy.shape

(577, 64, 64, 1)

In [17]:
save_model_as_json(sym_autoencoder,'CNNDSC_MINI')

In [18]:
#Getting images to test
def get_testing_images(filePath):
    test_images = read_dataset(filePath)
    test_images = np.array(test_images)
    test_images = test_images.astype('float32')/255.
    test_images = np.reshape(test_images, (test_images.shape[0],64,64,1))
    noisy_test_images = add_noise(test_images)
    print (noisy_test_images.shape)
    return (test_images,noisy_test_images)

In [19]:
test_images,noisy_test_images = get_testing_images('data/Test/')
noisy_test_images.shape

(120, 64, 64, 1)


(120, 64, 64, 1)

In [20]:
#Denoising using CNN symmetric Autoencoders
out_sym_autoencoder = sym_autoencoder.predict(noisy_test_images,verbose=1)
out_sym_autoencoder.shape

120/120 [==============================] - 7s     


(120, 64, 64, 1)

In [21]:
#Denoising using Autoencoders
out_autoencoder = autoencoder.predict(noisy_test_images,verbose=1)
out_autoencoder.shape

120/120 [==============================] - 1s     


(120, 64, 64, 1)

In [22]:
"""
display_images(test_images,10)
display_images(noisy_test_images,10)
display_images(out,10)
"""

'\ndisplay_images(test_images,10)\ndisplay_images(noisy_test_images,10)\ndisplay_images(out,10)\n'

In [47]:
from sklearn.metrics import mean_squared_error
def get_psnr(imageA,imageB):
    maxI = 1.0
    try:
        return 20*math.log10(maxI) - 10*math.log10(mean_squared_error(imageA.flatten(),imageB.flatten()))
    except:
        return 20*math.log10(maxI)

def get_psnr_result(x_test, out):
    psnr_sum = 0
    for i in range(out.shape[0]):
        psnr_sum += get_psnr(x_test[i].reshape(64,64),out[i].reshape(64,64))
        
    return 1.0*psnr_sum/out.shape[0];

def get_ssim_result(originalSet,noisySet):
    ssim_sum = 0
    originalSet = originalSet.reshape(originalSet.shape[0],64, 64, 1)
    noisySet = noisySet.reshape(noisySet.shape[0],64, 64, 1)
    for i in range(originalSet.shape[0]):
        ssim_sum += ssim(originalSet[i], noisySet[i],data_range=originalSet[i].max() - noisySet[i].min(), multichannel=True)
    return 1.0*ssim_sum/originalSet.shape[0]

In [24]:
def bm3d_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],64,64)
    noisy_image = noisy_image*255.0
    denoised = []
    for i in range(noisy_image.shape[0]):
        Basic_img = bm3d.BM3D_1st_step(noisy_image[i])
        Final_img = bm3d.BM3D_2nd_step(Basic_img, noisy_image[i])
        denoised.append(Final_img)
        print "Image " + str(i) + " denoised"
    return np.array(denoised)

In [25]:
def nlm_denoise(noisy_image):
    noisy_image = noisy_image.reshape(noisy_image.shape[0],64,64)
    noisy_image = noisy_image*255.0
    denoised = []
    
    for image in noisy_image:
        denoised_image = denoise_nl_means(image, 7, 11, 0.5)
        denoised.append(denoised_image)
        
    return np.array(denoised)

In [27]:
get_psnr_result(out_autoencoder,test_images)

14.700186873509635

In [28]:
get_psnr_result(out_sym_autoencoder,test_images)

15.174594201883929

In [31]:
import bm3d

In [52]:
print noisy_test_images.shape
bm3d_out = bm3d_denoise(noisy_test_images)

(120, 64, 64, 1)
Image 0 denoised
Image 1 denoised
Image 2 denoised
Image 3 denoised
Image 4 denoised
Image 5 denoised
Image 6 denoised
Image 7 denoised
Image 8 denoised
Image 9 denoised
Image 10 denoised
Image 11 denoised
Image 12 denoised
Image 13 denoised
Image 14 denoised
Image 15 denoised
Image 16 denoised
Image 17 denoised
Image 18 denoised
Image 19 denoised
Image 20 denoised
Image 21 denoised
Image 22 denoised
Image 23 denoised
Image 24 denoised
Image 25 denoised
Image 26 denoised
Image 27 denoised
Image 28 denoised
Image 29 denoised
Image 30 denoised
Image 31 denoised
Image 32 denoised
Image 33 denoised
Image 34 denoised
Image 35 denoised
Image 36 denoised
Image 37 denoised
Image 38 denoised
Image 39 denoised
Image 40 denoised
Image 41 denoised
Image 42 denoised
Image 43 denoised
Image 44 denoised
Image 45 denoised
Image 46 denoised
Image 47 denoised
Image 48 denoised
Image 49 denoised
Image 50 denoised
Image 51 denoised
Image 52 denoised
Image 53 denoised
Image 54 denoised
Ima

In [53]:
bm3d_out_norm = bm3d_out.astype('float32')/255.0

In [54]:
get_psnr_result(bm3d_out_norm,test_images)

17.799269401238046

In [39]:
nlm_out = nlm_denoise(noisy_test_images)
nlm_out = nlm_out.astype('float32')/255.0

/home/yeshwanth/anaconda2/lib/python2.7/site-packages/skimage/restoration/non_local_means.py:108: UserWarning: denoise_nl_means will default to multichannel=False in v0.15
  warn('denoise_nl_means will default to multichannel=False in v0.15')


In [40]:
get_psnr_result(nlm_out,test_images)

13.178280774231597

In [48]:
get_ssim_result(test_images,out_sym_autoencoder)

0.48315060087070411

In [49]:
get_ssim_result(test_images,out_autoencoder)

0.38761606919347613

In [50]:
get_ssim_result(test_images,bm3d_out_norm)

IndexError: index 10 is out of bounds for axis 0 with size 10

In [51]:
get_ssim_result(test_images,nlm_out)

0.28062358379392816

In [ ]:
#Ground Truth
display_images(test_images,10)

In [ ]:
#Noisy images
display_images(noisy_test_images,10

In [ ]:
#Autoencoder denoised images
display_images(out,10)

In [ ]:
#bm3d denoised images
display_images(bm3d_out_norm,10)

In [ ]:
#nlm denoised images
display_images(nlm_out,10)